# Guided Capstone Step 4. Pre-Processing and Training Data Development

**The Data Science Method**  


1.   Problem Identification 


2.   Data Wrangling 
  
 
3.   Exploratory Data Analysis   

4.   **Pre-processing and Training Data Development**  
 * Create dummy or indicator features for categorical variables
  * Standardize the magnitude of numeric features
  * Split into testing and training datasets
  * Apply scaler to the testing set
5.   Modeling 
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

**<font color='teal'> Start by loading the necessary packages as we did in step 3 and printing out our current working directory just to confirm we are in the correct project directory. </font>**

In [1]:
#load python packages
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
#os.getcwd()

**<font color='teal'>  Load the csv file you created in step 3, remember it should be saved inside your data subfolder and print the first five rows.</font>**

In [2]:
path="/Users/huluhulu/Desktop/Uni/Data Science/springboard/Guided Capstone/data"
os.chdir(path) 
os.getcwd()
df = pd.read_csv('step3_output.csv')
df.head()

,Name,state,summit_elev,vertical_drop,trams,fastEight,fastSixes,fastQuads,quad,triple,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
0,Alyeska Resort,Alaska,3939,2500,1,0.0,0,2,2,0,...,1610.0,113.0,150.0,60.0,669.0,65.0,85.0,150.0,550.0,1
1,Eaglecrest Ski Area,Alaska,2600,1540,0,0.0,0,0,0,0,...,640.0,60.0,45.0,44.0,350.0,47.0,53.0,90.0,0.0,1
2,Hilltop Ski Area,Alaska,2090,294,0,0.0,0,0,0,1,...,30.0,30.0,150.0,36.0,69.0,30.0,34.0,152.0,30.0,1
3,Arizona Snowbowl,Arizona,11500,2300,0,0.0,1,0,2,2,...,777.0,104.0,122.0,81.0,260.0,89.0,89.0,122.0,0.0,0
4,Sunrise Park Resort,Arizona,11100,1800,0,0.0,0,1,2,3,...,800.0,80.0,115.0,49.0,250.0,74.0,78.0,104.0,80.0,0


In [3]:
df[df['Name'].str.contains('Big Mountain')]

,Name,state,summit_elev,vertical_drop,trams,fastEight,fastSixes,fastQuads,quad,triple,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
151,Big Mountain Resort,Montana,6817,2353,0,0.0,0,3,2,6,...,3000.0,600.0,123.0,72.0,333.0,81.0,81.0,123.0,600.0,0


## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `state`. Add the dummies back to the dataframe and remove the original column for `state`. </font>**

Hint: you can see an example of how to execute this in Aiden's article on preprocessing [here](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-pre-processing-and-training-data-development-fd2d75182967). 

In [4]:
df['state'].nunique() #35
df.columns

Index(['Name', 'state', 'summit_elev', 'vertical_drop', 'trams', 'fastEight',
       'fastSixes', 'fastQuads', 'quad', 'triple', 'double', 'surface',
       'total_chairs', 'Runs', 'TerrainParks', 'LongestRun_mi',
       'SkiableTerrain_ac', 'Snow Making_ac', 'daysOpenLastYear', 'yearsOpen',
       'averageSnowfall', 'AdultWeekday', 'AdultWeekend', 'projectedDaysOpen',
       'NightSkiing_ac', 'clusters'],
      dtype='object')

In [5]:
df.shape


(330, 26)

In [6]:
df = pd.concat([df, pd.get_dummies(df['state'])], axis=1).drop(['state'], axis =1)
print(df.shape)
df.head()


(330, 60)


,Name,summit_elev,vertical_drop,trams,fastEight,fastSixes,fastQuads,quad,triple,double,...,Rhode Island,South Dakota,Tennessee,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,Alyeska Resort,3939,2500,1,0.0,0,2,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Eaglecrest Ski Area,2600,1540,0,0.0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0
2,Hilltop Ski Area,2090,294,0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,Arizona Snowbowl,11500,2300,0,0.0,1,0,2,2,1,...,0,0,0,0,0,0,0,0,0,0
4,Sunrise Park Resort,11100,1800,0,0.0,0,1,2,3,1,...,0,0,0,0,0,0,0,0,0,0


## Standardize the magnitude of numeric features

**<font color='teal'> Using sklearn preprocessing standardize the scale of the features of the dataframe except the name of the resort which we done't need in the dataframe for modeling, so it can be droppped here as well. Also, we want to hold out our response variable(s) so we can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as our response for scaling and modeling. Later we will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave them in the development dataframe. </font>**

In [7]:
# first we import the preprocessing package from the sklearn library
from sklearn import preprocessing

# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X = df.drop(['Name','AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = df.AdultWeekend

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = preprocessing.StandardScaler().fit(X)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled=scaler.transform(X) 

## Split into training and testing datasets

**<font color='teal'> Using sklearn model selection import train_test_split, and create a 75/25 split with the y = `AdultWeekend`. We will start by using the adult weekend ticket price as our response variable for modeling.</font>**

In [8]:
# Import the train_test_split function from the sklearn.model_selection utility.  
from sklearn.model_selection import train_test_split

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

Here we start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

# Guided Capstone Step 5. Modeling


This is the fifth step in the Data Science Method. In the previous steps you cleaned and prepared the datasets. Now it's time to get into the most exciting part: modeling! In this exercise, you'll build three different models and compare each model's performance. In the end, you'll choose the best model for demonstrating insights to Big Mountain management.



### **The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Fit Models with a Training Dataset

**<font color='teal'> Using sklearn, fit the model on your training dataset.</font>**

#### Model 1

In [9]:
#all first model set
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

Hint: you will have to use the `predict()` method here as it's used in this [DSM article](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-modeling-56b4233cad1b) about modeling. 

In [10]:
# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
y_pred = model.predict(X_test)
print(y_pred)

[ 58.03452034  88.38541578  78.68400049  75.71454267  69.28053467
  76.39294848  44.60727415  46.75858165  93.68396061  84.4643673
  50.527702    79.14867534  50.71678882  70.85587726  62.49329854
  93.38189115  42.28572497  73.1516585   61.94987631  40.80719333
 113.42571     36.90509822  53.73551506  49.44054151  48.00726007
  63.32992116  76.21623413  75.25745763  63.70124254  66.59113174
  34.58542863  59.20889029  80.36428138  57.08549194  75.48512681
  49.38156163  22.97032222  29.22040255  61.47088012  37.17328678
  48.43071958  63.27198003  86.69931472  72.3991289   72.74705615
  58.47898959  60.70380757 164.96402784  37.76322119  61.06498483
  68.35255583  60.42655722  53.31731275  43.83732919  59.25642617
  42.08488118  85.2770178   39.4712914  119.24752909 118.99668016
  56.12767406  53.69913559  41.81999853  81.06444154  64.58031414
  43.2547136   36.34513142  96.48910951  61.15479821  24.05381324
  68.08107714 146.2636211   57.97564108  62.40688962  75.20579898
  70.953797

## Review Model Outcomes — Iterate over additional models as needed

In [11]:
# You might want to use the explained_variance_score() and mean_absolute_error() metrics.
# To do so, you will need to import them from sklearn.metrics. 
# You can plug y_test and y_pred into the functions to evaluate the model
from sklearn.metrics import explained_variance_score,mean_absolute_error
print(explained_variance_score(y_test, y_pred))
mean_absolute_error(y_test, y_pred)

0.9410550802206951


4.867549167897311

**<font color='teal'> Print the intercept value from the linear model. </font>**

Hint: our linear regression model `lm` has an attribute `intercept_` for the intercept

In [12]:
lm.intercept_

10.32257885297232

**<font color='teal'> The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. The addition or subtraction of each of the coefficient values in the regression are numeric adjustments applied to the intercept to provide a particular observation's value for the resulting `AdultWeekend` value. Also, because we took the time to scale our x values in the training data, we can compare each of the coeeficients for the features to determine the feature importances. Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.</font>** 


Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative, but what we are looking for is the magnitude of impact on our response variable.

In [13]:
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
# pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])
coef_df = pd.DataFrame(abs(lm.coef_),X.columns, columns=['Coefficient'])  
coef_df.sort_values(by=['Coefficient'],ascending=False).head(10)

,Coefficient
Tennessee,19.175426
North Carolina,12.554420
Maryland,10.011996
West Virginia,9.729164
Montana,9.065093
Massachusetts,7.781670
New Mexico,7.127162
Idaho,6.854625
Virginia,6.722499
Ohio,6.508006


**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that we care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

Hint: Try to construct another model using exactly the steps we followed above. 

#### Model 2

In [14]:
df2 = pd.read_csv('step3_output.csv')
print(df2.head())
print(df2.shape)

                  Name    state  summit_elev  vertical_drop  trams  fastEight  \
0       Alyeska Resort   Alaska         3939           2500      1        0.0   
1  Eaglecrest Ski Area   Alaska         2600           1540      0        0.0   
2     Hilltop Ski Area   Alaska         2090            294      0        0.0   
3     Arizona Snowbowl  Arizona        11500           2300      0        0.0   
4  Sunrise Park Resort  Arizona        11100           1800      0        0.0   

   fastSixes  fastQuads  quad  triple  ...  SkiableTerrain_ac  Snow Making_ac  \
0          0          2     2       0  ...             1610.0           113.0   
1          0          0     0       0  ...              640.0            60.0   
2          0          0     0       1  ...               30.0            30.0   
3          1          0     2       2  ...              777.0           104.0   
4          0          1     2       3  ...              800.0            80.0   

   daysOpenLastYear  years

In [16]:
from sklearn import preprocessing
X2=df2.drop(['AdultWeekend','Name','state'], axis=1)
y=df2.AdultWeekend
y=y.ravel()
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y, test_size=0.25, random_state=1)

scaler2 = preprocessing.StandardScaler().fit(X2)
X_scaled2=scaler2.transform(X2) 

In [17]:
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error
lm2 = linear_model.LinearRegression()
model2 = lm2.fit(X_train2,y_train2)
y_pred2 = model2.predict(X_test2)
print(y_pred2)

[ 59.36161874  77.50923618  82.99938592  78.27600133  69.2219977
  80.85346732  36.82512257  48.15094872  88.83378264  87.14063447
  51.76176766  81.2683133   52.67149839  65.32036348  64.09581572
  90.21167745  43.76845502  77.74787908  64.27714581  42.70273529
 109.00830058  37.80049433  54.42595395  50.63095466  49.16276407
  61.1577395   68.61717867  67.38199187  61.32688785  65.41394631
  37.35745759  54.66355611  80.80402017  57.28060418  77.64261879
  54.2022703   24.13650078  30.06598783  53.31549335  38.02792787
  48.15477326  62.68799692  90.49807009  68.73943474  71.26535819
  52.81668401  62.25938921 166.63605467  31.00438104  63.0975135
  61.44030895  62.40036481  53.05524211  44.58972114  59.95109491
  41.52487654  84.93128255  32.31973518 122.39389129 115.61673479
  57.51575037  54.60221912  40.35583916  86.08598978  64.38197336
  48.2082399   38.06309513  94.74660232  57.17535755  27.56848156
  64.73520568 150.71687554  56.02221147  63.87863463  76.17149777
  68.5372589

In [18]:
from sklearn.metrics import explained_variance_score,mean_absolute_error
print(explained_variance_score(y_test2, y_pred2))
mean_absolute_error(y_test2, y_pred2)

0.9339426332708693


5.225538875500041

In [19]:
lm2.intercept_

1.5336178490427201

In [20]:
coef_df2 = pd.DataFrame(abs(lm2.coef_),X2.columns, columns=['Coefficient'])  
coef_df2.sort_values(by=['Coefficient'],ascending=False).head(10)

,Coefficient
clusters,11.457942
fastEight,4.116043
surface,1.492837
total_chairs,0.909377
trams,0.849570
AdultWeekday,0.827565
double,0.670241
LongestRun_mi,0.418906
fastQuads,0.249808
triple,0.082198


**<font color='teal'> When reviewing our new model coefficients, we see `summit_elev` is now in the number two spot. This is also difficult to change from a management prespective and highly correlated with `base_elev` and `vertical_drop`.  This time, rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

#### Model 3

In [21]:
df3 = pd.read_csv('step3_output.csv')
print(df3.head())
print(df3.shape)

                  Name    state  summit_elev  vertical_drop  trams  fastEight  \
0       Alyeska Resort   Alaska         3939           2500      1        0.0   
1  Eaglecrest Ski Area   Alaska         2600           1540      0        0.0   
2     Hilltop Ski Area   Alaska         2090            294      0        0.0   
3     Arizona Snowbowl  Arizona        11500           2300      0        0.0   
4  Sunrise Park Resort  Arizona        11100           1800      0        0.0   

   fastSixes  fastQuads  quad  triple  ...  SkiableTerrain_ac  Snow Making_ac  \
0          0          2     2       0  ...             1610.0           113.0   
1          0          0     0       0  ...              640.0            60.0   
2          0          0     0       1  ...               30.0            30.0   
3          1          0     2       2  ...              777.0           104.0   
4          0          1     2       3  ...              800.0            80.0   

   daysOpenLastYear  years

In [22]:
from sklearn import preprocessing
X3=df3.drop(['AdultWeekend','Name','state','summit_elev'], axis=1)
y=df3.AdultWeekend
y=y.ravel()
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y, test_size=0.25, random_state=1)

scaler3 = preprocessing.StandardScaler().fit(X3)
X_scaled3=scaler3.transform(X3) 

In [24]:
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error
lm3 = linear_model.LinearRegression()
model3 = lm3.fit(X_train3,y_train3)
y_pred3 = model3.predict(X_test3)
print(y_pred3)

[ 58.00681921  81.47115191  80.03324034  77.69947289  69.57687661
  78.72624797  37.83501404  48.43402622  89.26166973  88.02347469
  51.94360339  78.44874276  52.95750489  66.10513158  64.00091676
  90.56263953  42.07087334  75.61290516  62.46370141  43.04573346
 109.11043371  38.13339804  55.35002431  50.47646928  49.95564442
  61.99612728  66.80833068  70.80643421  62.45075822  65.04939355
  36.73411303  51.62854426  79.1585533   57.89019822  77.87053702
  54.0346103   25.00040597  29.83251116  54.72790754  39.14478773
  49.271908    63.02407207  90.34472424  68.81431778  69.61711089
  52.67689494  61.40714016 166.91524188  30.82462493  61.04986971
  62.75733243  61.0427192   53.13288022  45.49978719  60.65537386
  41.29797241  85.92187143  31.95599105 124.35003156 116.06966386
  58.27713355  52.34834117  40.02761379  87.38586614  65.0491867
  48.84546456  37.89502135  99.12471994  57.88021817  24.15330306
  64.64427336 151.36978131  56.8756284   63.43449604  76.76113931
  69.272520

In [25]:
from sklearn.metrics import explained_variance_score,mean_absolute_error
print(explained_variance_score(y_test3, y_pred3))
mean_absolute_error(y_test3, y_pred3)

0.9358117814573448


5.150579945762362

In [26]:
lm3.intercept_

8.086069136664321

In [27]:
coef_df3 = pd.DataFrame(abs(lm3.coef_),X3.columns, columns=['Coefficient'])  
coef_df3.sort_values(by=['Coefficient'],ascending=False).head(10)

,Coefficient
clusters,5.460975
fastEight,1.161100
surface,0.987887
AdultWeekday,0.839218
trams,0.813115
quad,0.501768
triple,0.493732
total_chairs,0.482589
fastSixes,0.370868
LongestRun_mi,0.342356


## Identify the Final Model

**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell — you will discuss this selection more in the next step of the guided casptone. </font>**

| Model | Explained Variance| Mean Absolute Error|Features Dropped|
| --- | --- | --- | --- |
| Model 1. | 0. |  |-|
| Model 2. | 0.| |'state'|
| Model 3. | 0. | |'state','summit_elev','base_elev'|

Model Selection:

In [ ]:
modelCompare = pd.DataFrame({'Model':['Model 1', 'Model 2', 'Model 3'], 
                             'Explained Variance': [0.9410550802206951, 0.9339426332708693, 0.9358117814573448],
                             'Mean Absolute Error': [4.867549167897311, 5.225538875500041, 5.150579945762362],
                             'Features Dropped': ['-', 'states', 'states, summit_elev, base_elev']})
modelCompare